In [2]:
import pandas as pd
from pygam import LinearGAM, s, f
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('assets/output/featureSelectionResult.csv')

In [4]:
X = df.drop(['date', 'visitor'], axis=1)
y = df['visitor']

In [6]:
selected_feat = ['sulfur_dioxide_min', 'carbon_monoxide_max', 'ozone_max', 'nitrogen_dioxide_max', 'fine_dust_pm10_max',                 'rainfall_max', 'probability of precipitation_min', 'humidity_min', 'highest temperature_max',                 'lowest temperature_min', 'wind speed_median', 'weekday_1.0' ,'weekday_2.0', 'weekday_3.0','weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'weekday_7.0',  'visitor']
df_selected = df[selected_feat]

X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

In [7]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv('assets/output/featureSelectionResult.csv')

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 상수 열 추가
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)

# GAM 모델 생성 및 학습
gam = LinearGAM()
gam.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = gam.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 평가: 결정 계수(R^2) 계산
r2_gam = r2_score(y_test, y_pred)
print(f"R^2 Score: {r2_gam}")

Mean Squared Error: 81659152.28296266
R^2 Score: 0.2735618808369892


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 데이터 로드
data = pd.read_csv('assets/output/featureSelectionResult.csv')

# Features와 Target 분리
X = df_selected.drop(['visitor'], axis=1)
y = df_selected['visitor']

# 다항 특성 추가
poly = PolynomialFeatures(degree=4)
X_poly = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 평가: 평균 제곱 오차 계산
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# 평가: 결정 계수(R^2) 계산
r2_lin = r2_score(X_test, y_test)
print(f"R^2 Score: {r2_lin}")

score = model.score(X_test, y_test)
print(f"score: {score}")

Mean Squared Error: 914723536.2566957
score: -8.072517541815914


In [8]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# 데이터 로드
data = pd.read_csv('assets/output/featureSelectionResult.csv')

# 데이터를 k개의 부분집합으로 나누는 k-fold 객체 생성
k = 5  # k-fold 교차 검증에서의 k 값 설정
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Features와 Target 분리
X = df.drop(['date', 'visitor'], axis=1)
y = df['visitor']

# 다항 특성 추가
poly = PolynomialFeatures(degree=7)
X_poly = poly.fit_transform(X)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# 모델 학습
model = LinearRegression()

# k-fold 교차 검증 수행
mse_scores = []  # 각 fold에서의 평균 제곱 오차(MSE)를 저장할 리스트
score_arr = []

for train_index, test_index in kf.split(X_poly):
    # 훈련 세트, 검증 세트 나누기
    X_train, X_test = X_poly[train_index], X_poly[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 모델 학습
    model.fit(X_train, y_train)

    # 검증 세트 예측
    y_pred = model.predict(X_test)

    # 검증 세트의 평균 제곱 오차 계산
    mse = mean_squared_error(y_test, y_pred)
    score = model.score(X_test, y_test)
    score_arr.append(score)
    mse_scores.append(mse)

# k-fold 교차 검증의 평균 MSE 계산
mean_mse = np.mean(mse_scores)
mean_score = np.mean(score_arr)

print(mean_score)

MemoryError: Unable to allocate 1.57 PiB for an array with shape (2468, 89356415775) and data type float64

In [3]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# 데이터 로드
df = pd.read_csv('assets/output/featureSelectionResult.csv')

def find_best_degree(X, y, min_degree, max_degree, min_k, max_k, random_state=42):
    results = []
    best_degree = None
    best_score = -np.inf
    best_k = None

    for degree in range(min_degree, max_degree+1):
        # 다항 특성 추가
        poly = PolynomialFeatures(degree=degree)
        X_poly = poly.fit_transform(X)

        for k in range(min_k, max_k+1):
            # 모델 학습
            model = LinearRegression()

            # k-fold 교차 검증 수행
            kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
            score_arr = []

            for train_index, test_index in kf.split(X_poly):
                # 훈련 세트, 검증 세트 나누기
                X_train, X_test = X_poly[train_index], X_poly[test_index]
                y_train, y_test = y[train_index], y[test_index]

                # 모델 학습
                model.fit(X_train, y_train)

                # 검증 세트의 점수 계산
                score = model.score(X_test, y_test)
                score_arr.append(score)

            # 평균 점수 계산
            mean_score = np.mean(score_arr)
            results.append((degree, k, mean_score))
            print(f"Degree: {degree}, K: {k}, Score: {mean_score}")

            # 현재 degree와 k 값의 점수가 최고 점수보다 높으면 업데이트
            if mean_score > best_score:
                best_degree = degree
                best_score = mean_score
                best_k = k
    return best_degree, best_score, best_k

X = df.drop(['date', 'visitor'], axis=1)
y = df['visitor']

min_degree = 2
max_degree = 6
min_k = 2
max_k = 10

best_degree, best_score, best_k = find_best_degree(X, y, min_degree, max_degree, min_k, max_k)

print(f"Best degree: {best_degree}")
print(f"Best score: {best_score}")
print(f"Best k: {best_k}")

Degree: 2, K: 2, Score: -1.4453290120723794
Degree: 2, K: 3, Score: -1.5500444006824747
Degree: 2, K: 4, Score: -2.5698713980269794
Degree: 2, K: 5, Score: -2.269611424698757
Degree: 2, K: 6, Score: -2.42114484270842
Degree: 2, K: 7, Score: -2.6019427159343893
Degree: 2, K: 8, Score: -2.516380485753828
Degree: 2, K: 9, Score: -2.6433477140108903
Degree: 2, K: 10, Score: -2.6515864031634773
Degree: 3, K: 2, Score: -2.744040871735411
Degree: 3, K: 3, Score: -0.7795631180669202
Degree: 3, K: 4, Score: -3.998184533635186
Degree: 3, K: 5, Score: -2.043004799453641
Degree: 3, K: 6, Score: -5.100733056045133
Degree: 3, K: 7, Score: -1.7761296344846766
Degree: 3, K: 8, Score: -1.6075872820962571
Degree: 3, K: 9, Score: -2.8519922668611106
Degree: 3, K: 10, Score: -2.761721710945369


MemoryError: Unable to allocate 173. GiB for an array with shape (2468, 9381251) and data type float64